In [66]:
import os
import sys
import numpy as np
from collections import defaultdict
import gmpy2
import copy
import logging

inputfilepath = 'C:\EMOLGUI\A_O_C\Day19.txt'
#inputfilepath = 'C:\EMOLGUI\A_O_C\Day19.sample.txt'

In [67]:
Blueprints = []
with open(inputfilepath,'r') as f:
    for line in f:
        blueprint = line.rstrip().split(':')[0].split()[1]
        robots = line.rstrip().split(':')[1].split('.')
        robotDict = {}
        for robot in robots:
            if not 'robot' in robot:
                print(f'Not found robot in {robot}')
                continue
            print(robot)
            #robot = robot.strip(' Each ')
            print(robot)
            robotType = robot.split(' robot costs ')[0].strip(' Eac').lstrip('h').lstrip()
            print(f'    {robotType}')
            robotCosts = robot.split(' robot costs ')[1].split(' and ')
            print(f'    {robotCosts}')
            costDict = {}
            for robotCost in robotCosts:
                costDict[robotCost.split()[1]] = int(robotCost.split()[0])
            robotDict[robotType] = costDict
        Blueprints.append(robotDict)            

 Each ore robot costs 4 ore
 Each ore robot costs 4 ore
    ore
    ['4 ore']
 Each clay robot costs 4 ore
 Each clay robot costs 4 ore
    clay
    ['4 ore']
 Each obsidian robot costs 4 ore and 8 clay
 Each obsidian robot costs 4 ore and 8 clay
    obsidian
    ['4 ore', '8 clay']
 Each geode robot costs 2 ore and 15 obsidian
 Each geode robot costs 2 ore and 15 obsidian
    geode
    ['2 ore', '15 obsidian']
Not found robot in 
 Each ore robot costs 4 ore
 Each ore robot costs 4 ore
    ore
    ['4 ore']
 Each clay robot costs 4 ore
 Each clay robot costs 4 ore
    clay
    ['4 ore']
 Each obsidian robot costs 3 ore and 19 clay
 Each obsidian robot costs 3 ore and 19 clay
    obsidian
    ['3 ore', '19 clay']
 Each geode robot costs 4 ore and 15 obsidian
 Each geode robot costs 4 ore and 15 obsidian
    geode
    ['4 ore', '15 obsidian']
Not found robot in 
 Each ore robot costs 4 ore
 Each ore robot costs 4 ore
    ore
    ['4 ore']
 Each clay robot costs 4 ore
 Each clay robot cos

In [68]:
Blueprints

[{'ore': {'ore': 4},
  'clay': {'ore': 4},
  'obsidian': {'ore': 4, 'clay': 8},
  'geode': {'ore': 2, 'obsidian': 15}},
 {'ore': {'ore': 4},
  'clay': {'ore': 4},
  'obsidian': {'ore': 3, 'clay': 19},
  'geode': {'ore': 4, 'obsidian': 15}},
 {'ore': {'ore': 4},
  'clay': {'ore': 4},
  'obsidian': {'ore': 2, 'clay': 8},
  'geode': {'ore': 3, 'obsidian': 9}},
 {'ore': {'ore': 3},
  'clay': {'ore': 3},
  'obsidian': {'ore': 2, 'clay': 12},
  'geode': {'ore': 2, 'obsidian': 10}},
 {'ore': {'ore': 3},
  'clay': {'ore': 4},
  'obsidian': {'ore': 4, 'clay': 6},
  'geode': {'ore': 3, 'obsidian': 16}},
 {'ore': {'ore': 2},
  'clay': {'ore': 4},
  'obsidian': {'ore': 3, 'clay': 19},
  'geode': {'ore': 4, 'obsidian': 8}},
 {'ore': {'ore': 4},
  'clay': {'ore': 3},
  'obsidian': {'ore': 2, 'clay': 20},
  'geode': {'ore': 3, 'obsidian': 9}},
 {'ore': {'ore': 2},
  'clay': {'ore': 4},
  'obsidian': {'ore': 3, 'clay': 20},
  'geode': {'ore': 2, 'obsidian': 16}},
 {'ore': {'ore': 3},
  'clay': {'ore':

In [48]:
production = {'ore': 1, 'clay': 0, 'obsidian': 0, 'geode': 0}
stock = {'ore': 0, 'clay': 0, 'obsidian': 0, 'geode': 0}

In [62]:
def validRobots(blueprint, production, stock, remainingTime):
    validList1 = []
    validList = []
    # List all robots that can be built based on material production
    for robotName, robotMat in blueprint.items():
        for productionReq in robotMat.keys():
            if production[productionReq] == 0:
                break
        else:
            validList1.append(robotName)

    # Discard robots that would increase material production for a material that already cannot be spent in the remaining time, or if the current production already matches maximum expenditure
    for robotName in validList1:
        if robotName == 'geode':
            validList.append(robotName)
            continue
        maxNeeded = 0
        currentProd = production[robotName]
        for anotherRobotName, robotMat in blueprint.items():
            if anotherRobotName == robotName:
                continue # Increase production to allow more of self is not justifiable
            if robotName in robotMat:
                maxNeeded = max(maxNeeded, robotMat[robotName])
        theorMaxNeeded = (maxNeeded - currentProd) * remainingTime
        if stock[robotName] < theorMaxNeeded:
            validList.append(robotName)


    return validList

In [ ]:
def validRobotsV2(blueprint, production, stock, remainingTime):
    validList1 = []
    validList = []
    # List all robots that can be built based on material production
    for robotName, robotMat in blueprint.items():
        for productionReq in robotMat.keys():
            if production[productionReq] == 0:
                break
        else:
            validList1.append(robotName)

    # Discard robots that would increase material production for a material that already cannot be spent in the remaining time, or if the current production already matches maximum expenditure
    needed = blueprint['geode']
    for resource, value in needed.items():
        deltaResource = (value - production[resource])*remainingTime - stock[resource]
    
    for robotName in validList1:
        if robotName == 'geode':
            validList.append(robotName)
            continue
        maxNeeded = 0
        currentProd = production[robotName]
        for anotherRobotName, robotMat in blueprint.items():
            if anotherRobotName == robotName:
                continue # Increase production to allow more of self is not justifiable
            if robotName in robotMat:
                maxNeeded = max(maxNeeded, robotMat[robotName])
        theorMaxNeeded = (maxNeeded - currentProd) * remainingTime
        if stock[robotName] < theorMaxNeeded:
            validList.append(robotName)


    return validList

In [50]:
def buildRobot(robot,stock):
    for stockType, stockVal in robot.items():
        if stockVal > stock[stockType]:
            return False
    for stockType, stockVal in robot.items():
        stock[stockType] -= stockVal
    return True

def increaseStock(stock, production):
    for stockType, stockVal in production.items():
        stock[stockType] += stockVal

In [51]:
myStock = {'ore': 20, 'clay': 6, 'obsidian': 0, 'geode': 0}
myRobot = {'ore': 6, 'clay': 3}
buildRobot(myRobot,myStock), buildRobot(myRobot,myStock), myStock

(True, True, {'ore': 8, 'clay': 0, 'obsidian': 0, 'geode': 0})

In [52]:
validRobots(Blueprints[0],production,stock,24)

['ore', 'clay']

In [53]:
def openGeodes(blueprint, production, stock, remainingTime):
    if remainingTime == 0:
        return stock['geode']

    geodeProd = [0]
    listOfRobots = validRobots(blueprint, production, stock, remainingTime)
    #print(f'Time remaining {remainingTime}, production {production}, valid robots {listOfRobots}')
    for robot in listOfRobots:
        thisStock = copy.deepcopy(stock)
        thisProduction = copy.deepcopy(production)
        thisRemainingTime = remainingTime
        robotReqs = blueprint[robot]
        while (not buildRobot(robotReqs, thisStock)) and (thisRemainingTime > 0):
            increaseStock(thisStock,thisProduction)
            thisRemainingTime -= 1
        if thisRemainingTime == 0:
            geodeProd.append(thisStock['geode'])
        else:
            increaseStock(thisStock,thisProduction)
            thisRemainingTime -= 1
            thisProduction[robot] += 1
            geodeProd.append(openGeodes(blueprint, thisProduction, thisStock, thisRemainingTime))
    return max(geodeProd)


In [41]:
timeLimit = 24
production = {'ore': 1, 'clay': 0, 'obsidian': 0, 'geode': 0}
stock = {'ore': 0, 'clay': 0, 'obsidian': 0, 'geode': 0}
print(Blueprints[0])

openGeodes(Blueprints[0], production, stock, timeLimit)

{'ore': {'ore': 4}, 'clay': {'ore': 4}, 'obsidian': {'ore': 4, 'clay': 8}, 'geode': {'ore': 2, 'obsidian': 15}}


1

In [42]:
results = [openGeodes(Blueprints[i], production, stock, timeLimit) for i in range(len(Blueprints))]

In [44]:
quality = 0
for index, result in enumerate(results):
    quality += (index+1)*result
quality

1404

#### QUESTION 2

In [69]:
timeLimit = 32
production = {'ore': 1, 'clay': 0, 'obsidian': 0, 'geode': 0}
stock = {'ore': 0, 'clay': 0, 'obsidian': 0, 'geode': 0}
print(Blueprints[0])

openGeodes(Blueprints[0], production, stock, timeLimit)

{'ore': {'ore': 4}, 'clay': {'ore': 4}, 'obsidian': {'ore': 4, 'clay': 8}, 'geode': {'ore': 2, 'obsidian': 15}}


21

In [70]:
timeLimit = 32
production = {'ore': 1, 'clay': 0, 'obsidian': 0, 'geode': 0}
stock = {'ore': 0, 'clay': 0, 'obsidian': 0, 'geode': 0}
print(Blueprints[1])

openGeodes(Blueprints[1], production, stock, timeLimit)

{'ore': {'ore': 4}, 'clay': {'ore': 4}, 'obsidian': {'ore': 3, 'clay': 19}, 'geode': {'ore': 4, 'obsidian': 15}}


7

In [71]:
timeLimit = 32
production = {'ore': 1, 'clay': 0, 'obsidian': 0, 'geode': 0}
stock = {'ore': 0, 'clay': 0, 'obsidian': 0, 'geode': 0}
print(Blueprints[2])

openGeodes(Blueprints[2], production, stock, timeLimit)

{'ore': {'ore': 4}, 'clay': {'ore': 4}, 'obsidian': {'ore': 2, 'clay': 8}, 'geode': {'ore': 3, 'obsidian': 9}}


40

In [72]:
21*7*40

5880